In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_all_matrixes.ipynb

,profile,age,gender,MS_HV,MS_type,EDSS,disease_duration,education,connectivity,degree,...,MFIS,SF36_mental,SF36_physical,DTI_FA,DTI_L1,DTI_MD,DTI_RX,RAW,FUNC,LS
ID,,,,,,,,,,,,,,,,,,,,,
FIS_001,none,45.506,female,MS,RR,1.5,16.772,secondary school,****,35.842105,...,8.0,42.97,54.45,"[[0.0, 0.439328825873252, 0.4625400010374732, ...","[[0.0, 0.001062246374944052, 0.001097384692440...","[[0.0, 0.000705840751585395, 0.000715541904089...","[[0.0, 0.0005276379438376123, 0.00052462053555...","[[0.0, 0.03552844351216702, 0.0006652241379404...",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_002,none,42.981,male,MS,RR,1.5,10.872,primary school,****,30.368421,...,4.0,60.25,53.15,"[[0.0, 0.4382497080416883, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001055207424665258, 0.0, 0.0, 0.0, 0....","[[0.0, 0.000703053658342256, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0005269767649653327, 0.0, 0.0, 0.0, 0...","[[0.0, 0.028578488896818185, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_004,none,52.239,female,MS,SP,6.0,21.828,graduate,****,30.736842,...,12.0,56.38,39.40,"[[0.0, 0.3322280520926731, 0.0, 0.0, 0.0, 0.37...","[[0.0, 0.001171813137463012, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0008873735393089818, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0007451537309838455, 0.0, 0.0, 0.0, 0...","[[0.0, 0.06991186050430216, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 1009.125, 0.0, 0.0, 0.0, 421.875, 0.0, ..."
FIS_005,none,36.672,male,MS,RR,1.0,11.619,post-graduate,****,27.736842,...,0.0,59.19,55.81,"[[0.0, 0.4078810391594139, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001044529380979277, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007169208696827965, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005531166160359597, 0.0, 0.0, 0.0, 0...","[[0.0, 0.02578800302151145, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 20.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
FIS_007,attention,31.233,female,MS,RR,2.0,13.928,post-graduate,****,30.184211,...,5.0,47.17,49.24,"[[0.0, 0.4314795444662643, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001069484424799801, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007114571565313628, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005324435254387877, 0.0, 0.0, 0.0, 0...","[[0.0, 0.035325089806942316, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 60.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


(98, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
FIS_001,0,0
FIS_002,0,0
FIS_004,0,0
FIS_005,0,0
FIS_007,1,0


(98, 2)

In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(65, 39)
(33, 39)
(65,)
(33,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', LogisticRegression(solver='liblinear'))
])

In [8]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 50, 3)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 100, 90)
MATRIXES = list(all_combinations(column_matrixes))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            


param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__penalty': PENALTY,
        'classify__C' : REGULATION_C,
}

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [ ]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=34.37078651685393 
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=34.37078651685393 
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=34.37078651685393 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=100.0 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=100.0 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=100.0 
[CV] reduce_dim__n_components=34, filter__colu

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.4s


[CV]  reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=81.08988764044945, score=0.6363636363636364, total=   1.8s
[CV] reduce_dim__n_components=13, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l2, classify__C=62.17977528089888 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'RAW', 'LS'), classify__penalty=l2, classify__C=67.74157303370787, score=0.5909090909090909, total=   3.3s
[CV]  reduce_dim__n_components=34, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW', 'LS'), classify__penalty=l1, classify__C=31.03370786516854, score=0.7142857142857143, total=   3.6s
[CV]  reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=81.08988764044945, score=0.

[CV]  reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__penalty=l2, classify__C=73.30337078651685, score=0.2727272727272727, total=   3.9s
[CV] reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'LS'), classify__penalty=l2, classify__C=93.32584269662922 
[CV] reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'LS'), classify__penalty=l2, classify__C=93.32584269662922 
[CV]  reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW'), classify__penalty=l2, classify__C=71.07865168539327, score=0.6818181818181818, total=   2.1s
[CV] reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'LS'), classify__penalty=l2, classify__C=93.32584269662922 
[CV]  reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__penalty=l2, classify__C

[CV]  reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__penalty=l2, classify__C=43.26966292134831, score=0.5909090909090909, total=   2.8s
[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__penalty=l1, classify__C=28.808988764044944 
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW', 'LS'), classify__penalty=l2, classify__C=77.75280898876404, score=0.7142857142857143, total=   1.4s
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW', 'LS'), classify__penalty=l2, classify__C=77.75280898876404, score=0.5454545454545454, total=   1.5s
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=49.943820224719104 
[CV] reduce_dim__n_components=43, filter__columns=None, com

...

[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW'), classify__penalty=l2, classify__C=1.0, score=0.6363636363636364, total=   2.7s
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_RX', 'LS'), classify__penalty=l2, classify__C=51.0561797752809 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('RAW',), classify__penalty=l1, classify__C=73.30337078651685, score=0.5909090909090909, total=   2.5s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__penalty=l2, classify__C=48.831460674157306 
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_RX', 'LS'), classify__penalty=l2, classify__C=51.0561797752809, score=0.5454545454545454, total=   2.0s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__penalty=l2, classify__C=48.8

[CV] reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__penalty=l2, classify__C=52.1685393258427 
[CV]  reduce_dim__n_components=40, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'RAW'), classify__penalty=l1, classify__C=100.0, score=0.7272727272727273, total=   1.5s
[CV]  reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=42.15730337078652, score=0.6818181818181818, total=   3.0s
[CV] reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__penalty=l2, classify__C=52.1685393258427 
[CV] reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l1, classify__C=2.1123595505617976 
[CV]  reduce_dim__n_components=40, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'RAW'), classify__penalty=l1, 

[CV]  reduce_dim__n_components=40, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'RAW'), classify__penalty=l2, classify__C=11.011235955056181, score=0.6666666666666666, total=   1.5s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA',), classify__penalty=l1, classify__C=1.0 
[CV]  reduce_dim__n_components=31, filter__columns=None, combining__columns=('LS',), classify__penalty=l2, classify__C=64.40449438202248, score=0.5909090909090909, total=   4.8s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA',), classify__penalty=l1, classify__C=1.0 
[CV]  reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA',), classify__penalty=l1, classify__C=1.0, score=0.7272727272727273, total=   3.1s
[CV] reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW', 'LS'), classify__penalty=l2, classify__C=97.7752808988764 
[CV]  reduce_dim__n_components=31

[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=25.471910112359552, score=0.5454545454545454, total=   1.3s
[CV] reduce_dim__n_components=34, filter__columns=None, combining__columns=('RAW',), classify__penalty=l1, classify__C=67.74157303370787 
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=25.471910112359552, score=0.4090909090909091, total=   1.3s
[CV] reduce_dim__n_components=34, filter__columns=None, combining__columns=('RAW',), classify__penalty=l1, classify__C=67.74157303370787 
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=25.471910112359552, score=0.7142857142857143, total=   1.7s
[CV]  reduce_dim__n_components=22, filter__columns=None, combining__columns=('DTI_RX',), classify__penalty=l1, classify__C=24.3595

[CV] reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_FA', 'RAW'), classify__penalty=l2, classify__C=27.69662921348315 
[CV]  reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_RX', 'LS'), classify__penalty=l2, classify__C=87.76404494382022, score=0.5714285714285714, total=   1.4s
[CV] reduce_dim__n_components=49, filter__columns=None, combining__columns=('DTI_FA', 'RAW'), classify__penalty=l2, classify__C=27.69662921348315 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('DTI_L1',), classify__penalty=l1, classify__C=66.62921348314607, score=0.7142857142857143, total=   2.8s
[CV] reduce_dim__n_components=28, filter__columns=None, combining__columns=('DTI_MD',), classify__penalty=l1, classify__C=74.41573033707866 
[CV]  reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_RX', 'LS'), classify__penalty=l2, classify__C=87.76404494382022, score=0.6363636363636364, total=   1.9s
[CV] reduce

In [ ]:
%run ../grid_results.ipynb